In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import os

BASE_DIR = "/content/drive/MyDrive/실전 프로젝트 2/train"

files = ["PPOdata.json", "RMdata.json", "RMlabel.json", "SFTdata.json", "SFTlabel.json"]

print("BASE_DIR 존재?", os.path.exists(BASE_DIR))
print("\n[폴더 안 파일 목록]")
print(os.listdir(BASE_DIR)[:50])

print("\n[필수 파일 존재 여부]")
for f in files:
    path = os.path.join(BASE_DIR, f)
    print(f"{f:12s} ->", "OK" if os.path.exists(path) else "MISSING", "|", path)


BASE_DIR 존재? True

[폴더 안 파일 목록]
['SFTdata.json', 'RMdata.json', 'PPOdata.json', 'RMlabel.json', 'SFTlabel.json']

[필수 파일 존재 여부]
PPOdata.json -> OK | /content/drive/MyDrive/실전 프로젝트 2/train/PPOdata.json
RMdata.json  -> OK | /content/drive/MyDrive/실전 프로젝트 2/train/RMdata.json
RMlabel.json -> OK | /content/drive/MyDrive/실전 프로젝트 2/train/RMlabel.json
SFTdata.json -> OK | /content/drive/MyDrive/실전 프로젝트 2/train/SFTdata.json
SFTlabel.json -> OK | /content/drive/MyDrive/실전 프로젝트 2/train/SFTlabel.json


In [5]:
# 2) JSON 로드 + data_id 매칭(조인 가능) 확인
#   - RMdata <-> RMlabel
#   - SFTdata <-> SFTlabel
import json

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

RMdata_path  = os.path.join(BASE_DIR, "RMdata.json")
RMlabel_path = os.path.join(BASE_DIR, "RMlabel.json")
SFTdata_path  = os.path.join(BASE_DIR, "SFTdata.json")
SFTlabel_path = os.path.join(BASE_DIR, "SFTlabel.json")

rm_data  = load_json(RMdata_path)
rm_label = load_json(RMlabel_path)
sft_data  = load_json(SFTdata_path)
sft_label = load_json(SFTlabel_path)

# 실제 레코드 리스트
rm_data_list  = rm_data["data_info"]
rm_label_list = rm_label["data_info"]
sft_data_list  = sft_data["data_info"]
sft_label_list = sft_label["data_info"]

def id_set(records):
    return set([r["data_id"] for r in records])

rm_data_ids  = id_set(rm_data_list)
rm_label_ids = id_set(rm_label_list)
sft_data_ids  = id_set(sft_data_list)
sft_label_ids = id_set(sft_label_list)

def check_match(name, ids_a, ids_b):
    inter = ids_a & ids_b
    only_a = ids_a - ids_b
    only_b = ids_b - ids_a

    print(f"\n==== {name} data_id 매칭 체크 ====")
    print("A 개수:", len(ids_a))
    print("B 개수:", len(ids_b))
    print("교집합:", len(inter))
    print("A에만 있음:", len(only_a))
    print("B에만 있음:", len(only_b))

    # 샘플 몇 개만 보여주기
    if len(only_a) > 0:
        print("A-only 샘플 5개:", list(only_a)[:5])
    if len(only_b) > 0:
        print("B-only 샘플 5개:", list(only_b)[:5])

check_match("RM (RMdata vs RMlabel)", rm_data_ids, rm_label_ids)
check_match("SFT (SFTdata vs SFTlabel)", sft_data_ids, sft_label_ids)



==== RM (RMdata vs RMlabel) data_id 매칭 체크 ====
A 개수: 26408
B 개수: 26408
교집합: 26408
A에만 있음: 0
B에만 있음: 0

==== SFT (SFTdata vs SFTlabel) data_id 매칭 체크 ====
A 개수: 10580
B 개수: 10580
교집합: 10580
A에만 있음: 0
B에만 있음: 0


In [6]:
# 3) 3개만 샘플로 확인
#    - RM: data에서 question, label에서 ranking 들어있는지
#    - SFT: data에서 question, label에서 answer 들어있는지

# RM: data_id 하나 골라서 data/label 둘 다에서 내용 출력
sample_rm_id = next(iter(rm_data_ids & rm_label_ids))
rm_data_map  = {r["data_id"]: r for r in rm_data_list}
rm_label_map = {r["data_id"]: r for r in rm_label_list}

print("\n[RM 조인 샘플 data_id]", sample_rm_id)
print("RMdata.question:", rm_data_map[sample_rm_id].get("question"))

# answer01에 ranking 있는지 확인
ans01 = rm_label_map[sample_rm_id].get("answer01", {})
print("RMlabel.answer01 keys:", list(ans01.keys()))
print("RMlabel.answer01.ranking:", ans01.get("ranking"))

# SFT: data_id 하나 골라서 data/label 둘 다에서 내용 출력
sample_sft_id = next(iter(sft_data_ids & sft_label_ids))
sft_data_map  = {r["data_id"]: r for r in sft_data_list}
sft_label_map = {r["data_id"]: r for r in sft_label_list}

print("\n[SFT 조인 샘플 data_id]", sample_sft_id)
print("SFTdata.question:", sft_data_map[sample_sft_id].get("question"))

ans = sft_label_map[sample_sft_id].get("answer", {})
print("SFTlabel.answer keys:", list(ans.keys()))
print("SFTlabel.answer.contents (앞 120자):", (ans.get("contents","")[:120] + "...") )



[RM 조인 샘플 data_id] 110a52ec-6167-4ef9-b77c-156a31e5537d
RMdata.question: "일본어 공부를 하고 있습니다."를 영어로 알려주세요.
RMlabel.answer01 keys: ['answer_count', 'ranking', 'contents']
RMlabel.answer01.ranking: 1

[SFT 조인 샘플 data_id] f3865320-e86e-453a-a1a3-ba585f6b5860
SFTdata.question: 노동 시장에서의 혁신과 창업은 어떻게 지원되고 장려되나요?
SFTlabel.answer keys: ['answer_count', 'contents']
SFTlabel.answer.contents (앞 120자): 노동 시장에서의 혁신과 창업은 대부분의 국가에서 적극적으로 지원되고 장려됩니다. 이는 법적, 정책적, 금융적 측면에서 이루어집니다. 여러 방식으로 지원되는데, 그중 일부는 다음과 같습니다.

1. 법적 지원: 법적인...


In [7]:
import os, json, re, random
import pandas as pd
import numpy as np

BASE_DIR = "/content/drive/MyDrive/실전 프로젝트 2/train"

RMdata_path  = os.path.join(BASE_DIR, "RMdata.json")
RMlabel_path = os.path.join(BASE_DIR, "RMlabel.json")

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

rm_data  = load_json(RMdata_path)
rm_label = load_json(RMlabel_path)

rm_data_list  = rm_data["data_info"]
rm_label_list = rm_label["data_info"]

rm_data_map  = {r["data_id"]: r for r in rm_data_list}
rm_label_map = {r["data_id"]: r for r in rm_label_list}

common_ids = sorted(set(rm_data_map.keys()) & set(rm_label_map.keys()))

print("RMdata 개수:", len(rm_data_map))
print("RMlabel 개수:", len(rm_label_map))
print("교집합(분석 대상):", len(common_ids))
print("예시 id:", common_ids[0])


RMdata 개수: 26408
RMlabel 개수: 26408
교집합(분석 대상): 26408
예시 id: 0007133d-604f-4769-9544-7260d4b71843


In [8]:
# --- feature 함수들 ---
AVOID_KW = [
    "잘 모르", "정확", "확실", "죄송", "도움이 되", "전문가", "의사", "상담",
    "확인해", "찾아보", "정보가 부족", "AI", "모르겠", "판단하기 어렵"
]

numlist_pat = re.compile(r'(^|\n)\s*\d+\s*[\.\)]')   # 1. 2) 같은 번호 목록
bullet_pat  = re.compile(r'(^|\n)\s*[-•\*]\s+')      # - • * 목록
multi_space_pat = re.compile(r'\s+')

def sentence_repeat_rate(text: str) -> float:
    """아주 간단한 반복률: 문장/줄 단위로 중복 비율"""
    if not text:
        return 0.0
    # 줄바꿈/마침표 기준으로 대충 문장 분해
    parts = re.split(r'[\n\.!?]+', text)
    parts = [multi_space_pat.sub(" ", p).strip() for p in parts]
    parts = [p for p in parts if len(p) >= 6]  # 너무 짧은 조각 제외
    if len(parts) <= 1:
        return 0.0
    unique = len(set(parts))
    return 1.0 - (unique / len(parts))

def extract_features(ans: str) -> dict:
    if ans is None:
        ans = ""
    s = ans.strip()
    char_len = len(s)
    word_len = len(s.split())
    newline_cnt = s.count("\n")
    numlist_cnt = len(numlist_pat.findall(s))
    bullet_cnt  = len(bullet_pat.findall(s))
    avoid_flag  = int(any(k in s for k in AVOID_KW))
    rep_rate    = sentence_repeat_rate(s)
    return {
        "char_len": char_len,
        "word_len": word_len,
        "newline_cnt": newline_cnt,
        "numlist_cnt": numlist_cnt,
        "bullet_cnt": bullet_cnt,
        "avoid_flag": avoid_flag,
        "repeat_rate": rep_rate
    }

# --- long dataframe 만들기 ---
rows = []
invalid_ids = []

for did in common_ids:
    q = rm_data_map[did].get("question", "")
    cat = rm_data_map[did].get("data_category", {})
    main = cat.get("main", None) if isinstance(cat, dict) else None
    middle = cat.get("middle", None) if isinstance(cat, dict) else None

    lab = rm_label_map[did]

    ranks_seen = []
    for i in range(1, 6):
        key = f"answer0{i}"
        aobj = lab.get(key, {})
        rnk = aobj.get("ranking", None)
        txt = aobj.get("contents", "")
        ranks_seen.append(rnk)

        feats = extract_features(txt)
        rows.append({
            "data_id": did,
            "rank": rnk,
            "answer_idx": i,
            "question": q,
            "main": main,
            "middle": middle,
            "answer": txt,
            **feats
        })

    # ranking 완전성 체크 (1~5가 정확히 1번씩 있는지)
    if sorted(ranks_seen) != [1,2,3,4,5]:
        invalid_ids.append((did, ranks_seen))

df = pd.DataFrame(rows)

print("long df shape:", df.shape)
print("rank 값 분포:\n", df["rank"].value_counts(dropna=False).sort_index())
print("\n랭킹 완전성(1~5 모두 + 중복없음) 깨진 data_id 개수:", len(invalid_ids))
if invalid_ids:
    print("예시 3개:", invalid_ids[:3])


long df shape: (132040, 14)
rank 값 분포:
 rank
1    26408
2    26408
3    26408
4    26408
5    26408
Name: count, dtype: int64

랭킹 완전성(1~5 모두 + 중복없음) 깨진 data_id 개수: 0


In [9]:
# rank별 요약 통계
metrics = ["char_len", "word_len", "newline_cnt", "numlist_cnt", "bullet_cnt", "avoid_flag", "repeat_rate"]

summary = df.groupby("rank")[metrics].agg(["count", "mean", "median"])
display(summary)

# rank1 vs rank5 비교 (효과크기 Cohen's d)
def cohens_d(a, b):
    a = np.asarray(a); b = np.asarray(b)
    a = a[~np.isnan(a)]; b = b[~np.isnan(b)]
    if len(a) < 2 or len(b) < 2:
        return np.nan
    s = np.sqrt(((a.std(ddof=1)**2) + (b.std(ddof=1)**2)) / 2)
    if s == 0:
        return 0.0
    return (a.mean() - b.mean()) / s

df1 = df[df["rank"] == 1]
df5 = df[df["rank"] == 5]

effect_rows = []
for m in metrics:
    d = cohens_d(df1[m].values, df5[m].values)
    effect_rows.append({
        "metric": m,
        "mean_rank1": float(np.nanmean(df1[m])),
        "mean_rank5": float(np.nanmean(df5[m])),
        "cohens_d(1-5)": float(d)
    })

effect_df = pd.DataFrame(effect_rows).sort_values("cohens_d(1-5)", ascending=False)
display(effect_df)

# 같은 질문에서 rank1 vs rank5 비교 3개
import random
sample_ids = random.sample(common_ids, 3)

for did in sample_ids:
    sub = df[df["data_id"] == did].copy()
    q = sub["question"].iloc[0]
    a1 = sub[sub["rank"] == 1]["answer"].iloc[0]
    a5 = sub[sub["rank"] == 5]["answer"].iloc[0]
    print("\n" + "="*80)
    print("data_id:", did)
    print("Q:", q)
    print("\n[rank1]\n", a1[:600], ("..." if len(a1) > 600 else ""))
    print("\n[rank5]\n", a5[:600], ("..." if len(a5) > 600 else ""))


char_len                    word_len                    newline_cnt  \
        count        mean median    count        mean median       count   
rank                                                                       
1       26408  588.385035  639.0    26408  139.105650  151.0       26408   
2       26408  606.999697  639.0    26408  142.818994  151.0       26408   
3       26408  587.611027  624.0    26408  138.352204  147.0       26408   
4       26408  571.881513  608.0    26408  134.670819  143.0       26408   
5       26408  549.986557  582.0    26408  129.245039  137.0       26408   

                      numlist_cnt  ...        bullet_cnt                   \
          mean median       count  ... median      count      mean median   
rank                               ...                                      
1     9.085883   10.0       26408  ...    4.0      26408  0.193426    0.0   
2     9.506513   10.0       26408  ...    4.0      26408  0.200129    0.0   
3     9.156089   10.0       26408  ...    4.0      26408  0.163132    0.0   
4     8.920933   10.0       26408  ...    4.0      26408  0.166692    0.0   
5     8.532869   10.0       26408  ...    3.0      26408  0.159762    0.0   

     avoid_flag                  repeat_rate                   
          count      mean median       count      mean median  
rank                                                           
1         26408  0.308770    0.0       26408  0.004847    0.0  
2         26408  0.313125    0.0       26408  0.004840    0.0  
3         26408  0.303317    0.0       26408  0.004364    0.0  
4         26408  0.306574    0.0       26408  0.004751    0.0  
5         26408  0.294040    0.0       26408  0.004345    0.0  

[5 rows x 21 columns]

,metric,mean_rank1,mean_rank5,cohens_d(1-5)
1,word_len,139.105650,129.245039,0.133417
0,char_len,588.385035,549.986557,0.116857
3,numlist_cnt,3.158967,2.899197,0.083365
2,newline_cnt,9.085883,8.532869,0.076761
5,avoid_flag,0.308770,0.294040,0.032105
4,bullet_cnt,0.193426,0.159762,0.025480
6,repeat_rate,0.004847,0.004345,0.012291



data_id: 4185b87d-f6f2-4244-8fb7-685b3bf88be4
Q: 갤럭시 탭 화면이 이상한 상황에 대한 해결 방법을 알려주세요.

[rank1]
 갤럭시 탭의 화면이 이상한 상황에 대한 몇 가지 해결 방법을 안내해 드리겠습니다.

1. 화면 재부팅: 전원 버튼을 길게 눌러 "전원 끄기" 옵션을 선택한 후 다시 전원을 켜 보세요. 이렇게 하면 잠시 동안 장치의 일시적인 문제가 해결될 수 있습니다.

2. 배터리 재부착: 갤럭시 탭의 배터리를 제거하고 다시 재부 착해 보세요. 이렇게 하면 배터리와 연결된 접촉 부분에 이물질이나 먼지가 있어서 생긴 문제를 해결할 수 있습니다.

3. 공장 초기화: 화면에 표시되는 내용이 완전히 이상한 경우, 공장 초기화를 시도해 볼 수 있습니다. 이 작업은 모든 데이터와 설정을 삭제하므로, 중요한 정보는 미리 백업해 두시기 바랍니다. 공장 초기화는 일반적으로 설정 메뉴에서 "백업 및 재설정" 또는 "시스템" 항목을 찾아 수행할 수 있습니다.

4. 전문가 상담: 위의 방법들이 문제를 해결하지 못하거나, 화면에 심각한 물리적 손상이 있는 경우에는 공인 서비스 센터나 전문가에게 상담을 받아 보는 것이 좋습니다. 그들은 더 깊은 수리 작업이나 부품 교체를 해 줄 수 있습니다.

이러한 방법들을 시도해 보세요. 그러나 상황에 따라 결과가 달라질 수 있으므로, 주의 깊게 해결 방법을 ...

[rank5]
 갤럭시 탭의 화면이 이상한 상황에 대한 몇 가지 해결 방법을 안내해 드리겠습니다.

1. 화면을 재부팅해 보세요. 가끔씩 장치가 일시적인 문제로 인해 화면에 이상한 현상이 발생할 수 있습니다. 장치를 재부팅하면 이러한 문제가 해결될 수 있습니다. 전원 버튼을 길게 누르고 나오는 메뉴에서 "재부팅" 또는 "다시 시작" 옵션을 선택하세요.

2. 소프트웨어 업데이트를 확인해 보세요. 갤럭시 탭에는 정기적으로 소프트웨어 업데이트가 제공됩니다. 업데이트를 통해 장치의 안정성과 성능이 향상될 수 있습니다. 설정 앱을 열고 

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter

CONTROL_CHARS = re.compile(r"[\x00-\x08\x0b\x0c\x0e-\x1f]")  # \t,\n,\r 제외한 제어문자

def normalize_question(text: str) -> str:
    if text is None:
        return ""
    text = CONTROL_CHARS.sub("", text)
    text = text.replace("\t", " ")
    text = text.strip()
    text = re.sub(r"[ ]{2,}", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text

def normalize_answer(text: str) -> str:
    if text is None:
        return ""
    text = CONTROL_CHARS.sub("", text)
    text = text.strip()
    text = re.sub(r"\n{4,}", "\n\n", text)
    return text


In [12]:
df["question_norm"] = df["question"].apply(normalize_question)
df["answer_norm"]   = df["answer"].apply(normalize_answer)


In [13]:
# SFT: data_id 교집합 만들기 (SFTdata vs SFTlabel)

sft_data_ids  = set(sft_data_map.keys())
sft_label_ids = set(sft_label_map.keys())

common_ids_sft = sorted(list(sft_data_ids & sft_label_ids))

print("SFT data ids :", len(sft_data_ids))
print("SFT label ids:", len(sft_label_ids))
print("SFT common   :", len(common_ids_sft))
print("SFT only data:", len(sft_data_ids - sft_label_ids))
print("SFT only label:", len(sft_label_ids - sft_data_ids))


SFT data ids : 10580
SFT label ids: 10580
SFT common   : 10580
SFT only data: 0
SFT only label: 0


In [14]:
sft_rows = []
for did in common_ids_sft:
    q = sft_data_map[did].get("question","")
    cat = sft_data_map[did].get("data_category", {})
    main = cat.get("main") if isinstance(cat, dict) else None
    middle = cat.get("middle") if isinstance(cat, dict) else None
    ans = sft_label_map[did].get("answer", {}).get("contents","")

    sft_rows.append({
        "data_id": did,
        "main": main,
        "middle": middle,
        "question": q,
        "answer": ans
    })

sft_df = pd.DataFrame(sft_rows)
sft_df["question_norm"] = sft_df["question"].apply(normalize_question)
sft_df["answer_norm"]   = sft_df["answer"].apply(normalize_answer)

print("SFT df:", sft_df.shape)


SFT df: (10580, 7)


In [20]:
PPO_PATH = "/content/drive/MyDrive/실전 프로젝트 2/train/PPOdata.json"

with open(PPO_PATH, "r", encoding="utf-8") as f:
    ppo_raw = json.load(f)

ppo_list = ppo_raw["data_info"]
ppo_data_map = {x["data_id"]: x for x in ppo_list}

ppo_rows = []
for did, rec in ppo_data_map.items():
    q = rec.get("question", "")
    cat = rec.get("data_category", {})
    main = cat.get("main") if isinstance(cat, dict) else None
    middle = cat.get("middle") if isinstance(cat, dict) else None

    ppo_rows.append({
        "data_id": did,
        "main": main,
        "middle": middle,
        "question": q
    })

ppo_df = pd.DataFrame(ppo_rows)
ppo_df["question_norm"] = ppo_df["question"].apply(normalize_question)
print("PPO df:", ppo_df.shape)


PPO df: (25443, 5)


In [25]:
def basic_len_stats(series, name):
    lens = series.fillna("").astype(str).apply(len)
    qs = np.percentile(lens, [50, 90, 95, 99, 99.5, 99.9])
    print(f"\n[{name}] char length percentiles")
    print("p50 p90 p95 p99 p99.5 p99.9 =", qs.astype(int))
    return lens

rm_ans_len  = basic_len_stats(df["answer_norm"], "RM answer_norm")
sft_ans_len = basic_len_stats(sft_df["answer_norm"], "SFT answer_norm")
sft_q_len   = basic_len_stats(sft_df["question_norm"], "SFT question_norm")
ppo_q_len   = basic_len_stats(ppo_df["question_norm"], "PPO question_norm")



[RM answer_norm] char length percentiles
p50 p90 p95 p99 p99.5 p99.9 = [ 618  884  978 1493 1697 2155]

[SFT answer_norm] char length percentiles
p50 p90 p95 p99 p99.5 p99.9 = [ 649  895  960 1094 1162 1290]

[SFT question_norm] char length percentiles
p50 p90 p95 p99 p99.5 p99.9 = [29 40 45 57 63 79]

[PPO question_norm] char length percentiles
p50 p90 p95 p99 p99.5 p99.9 = [ 30  44  48  61  75 138]


In [26]:
# === SFT 품질 EDA: 회피문구 비율 + 길이 outlier 샘플링 ===
import numpy as np
import pandas as pd

AVOID_KW = [
    "잘 모르", "정확", "확실", "죄송", "도움이 되", "전문가", "의사", "상담",
    "확인해", "찾아보", "정보가 부족", "AI", "모르겠", "판단하기 어렵"
]

def avoid_flag(text: str) -> int:
    s = (text or "")
    return int(any(k in s for k in AVOID_KW))

# 길이/회피 feature
sft_df["ans_char_len"] = sft_df["answer_norm"].fillna("").astype(str).apply(len)
sft_df["avoid_flag"]   = sft_df["answer_norm"].fillna("").astype(str).apply(avoid_flag)

print("SFT rows:", len(sft_df))
print("회피문구 포함 비율:", round(sft_df["avoid_flag"].mean()*100, 2), "%")

# 길이 분위수
qs = np.percentile(sft_df["ans_char_len"], [50,90,95,99,99.5,99.9]).astype(int)
print("answer_norm char_len percentiles (p50 p90 p95 p99 p99.5 p99.9):", qs)

# outlier 샘플 출력: 긴 것 TOP 5 + 짧은 것 TOP 5
print("\n=== 긴 답변 TOP 5 ===")
for i, r in sft_df.sort_values("ans_char_len", ascending=False).head(5).iterrows():
    print("\n--- len:", r["ans_char_len"], "data_id:", r["data_id"], "---")
    print("Q:", r["question_norm"][:200])
    print("A:", r["answer_norm"][:800], ("..." if r["ans_char_len"]>800 else ""))

print("\n=== 짧은 답변 TOP 5 ===")
for i, r in sft_df.sort_values("ans_char_len", ascending=True).head(5).iterrows():
    print("\n--- len:", r["ans_char_len"], "data_id:", r["data_id"], "---")
    print("Q:", r["question_norm"][:200])
    print("A:", r["answer_norm"][:800], ("..." if r["ans_char_len"]>800 else ""))


SFT rows: 10580
회피문구 포함 비율: 24.05 %
answer_norm char_len percentiles (p50 p90 p95 p99 p99.5 p99.9): [ 649  895  960 1094 1162 1290]

=== 긴 답변 TOP 5 ===

--- len: 1721 data_id: c083a13f-91fc-4bae-9a28-13219dc3a72e ---
Q: 의사 결정에서 공평성과 정의는 어떻게 고려되어야 하나요?
A: 의사 결정에서 공평성과 정의는 중요한 요소로 고려되어야 합니다. 다음과 같은 방법으로 고려될 수 있습니다:

1. 균등한 대우: 공평성은 모든 개인이 동등하게 대우받는 것을 의미합니다. 의사 결정에 있어서는 모든 이해관계자에 대해 공평하게 대우해야 합니다. 어떤 개인이나 그룹이 불필요하게 혜택을 받거나 불이익을 갖지 않도록 노력해야 합니다.

2. 잠재적 평등: 잠재적 평등은 사회적 불평등을 보완하기 위해 취해지는 조치입니다. 의사 결정과정에서는 기존의 불평등을 해소하기 위해 적절한 조치를 취할 필요가 있습니다. 이는 많은 자원과 기회가 약자나 취약계층에게 더 공평하게 분배되도록 하는 것을 의미합니다.

3. 정의로운 분배: 정의로운 분배는 자원, 혜택, 기회 등을 어떻게 분배할지에 대한 원칙을 제시합니다. 의사 결정과정에서는 모든 이해관계자의 필요와 권리를 적절하게 고려하여 자원을 분배해야 합니다. 이는 사회적 정의와 더불어 판단하고 행동해야 하는 가치입니다.

4. 외부자의 참여: 공평성과 정의를 고려하기 위해서는 의사 결정에 영향을 주는 외부자들의 참여와 의견을 수렴하는 것이 중요합니다. 다양한 이해관계자들의 의견을 듣고 다양한 관점을 고려하는 것이 결정의 공정성을 높일 수 있습니다.

5. 법적, 도덕적 가이드라인 준수: 공평성과 정의를 고려하기 위해서는 법적인 가이드라인과 도덕적인 원칙을 준수해야 합니다. 법과 도덕은 사회적 규범을 제시하며, 이를 준수함으로써 공평성과 정의를 확보할 수 있습니다.

디도

In [28]:
import re, pandas as pd

numlist_pat = re.compile(r'(^|\n)\s*\d+\s*[\.\)]')
bullet_pat  = re.compile(r'(^|\n)\s*[-•\*]\s+')

def sft_structure_metrics(text: str):
    s = (text or "")
    return {
        "char_len": len(s),
        "newline_cnt": s.count("\n"),
        "numlist_cnt": len(numlist_pat.findall(s)),
        "bullet_cnt":  len(bullet_pat.findall(s)),
    }

sft_struct = sft_df["answer_norm"].apply(sft_structure_metrics).apply(pd.Series)
desc = sft_struct.describe(percentiles=[.5, .9, .95, .99])
print(desc.loc[["mean","50%","90%","95%","99%"]])

AVOID_KW = [
    "잘 모르", "정확", "확실", "죄송", "도움이 되", "전문가", "의사", "상담",
    "확인해", "찾아보", "정보가 부족", "모르겠", "판단하기 어렵"
]
def has_avoid(text: str) -> int:
    t = (text or "")
    return int(any(k in t for k in AVOID_KW))

sft_df["avoid_flag"] = sft_df["answer_norm"].apply(has_avoid)
print("SFT avoid ratio:", sft_df["avoid_flag"].mean())

def head_prefix(text, n=50):
    t = (text or "").strip().replace("\n"," ")
    return t[:n]

prefixes = sft_df["answer_norm"].apply(lambda x: head_prefix(x, 50))
print(prefixes.value_counts().head(20))

print(pd.concat([
    sft_struct.mean().rename("mean"),
    sft_struct.quantile(0.50).rename("50%"),
    sft_struct.quantile(0.90).rename("90%"),
    sft_struct.quantile(0.95).rename("95%"),
    sft_struct.quantile(0.99).rename("99%")
], axis=1).T)



         char_len  newline_cnt  numlist_cnt  bullet_cnt
mean   603.761437     8.702552     2.810397    0.170227
50%    649.000000    10.000000     3.000000    0.000000
90%    895.000000    14.000000     6.000000    0.000000
95%    960.000000    16.000000     7.000000    0.000000
99%   1094.000000    22.000000    10.000000    7.000000
SFT avoid ratio: 0.23686200378071834
answer_norm
죄송합니다. 특정 개인에 대한 질문에는 답변하기 어려우며, 개인정보 보호와 윤리적인 이유로    119
죄송합니다. 저는 인공지능 챗봇으로 일반적인 지식을 통해 학습되어, 주관적인 판단이 개입되     11
양자 상호작용은 양자역학에서 다양한 방법으로 묘사될 수 있습니다. 다음은 주요한 세 가지       2
미용실에서의 헤어트리트먼트와 집에서 하는 헤어팩의 주요 차이점은 다음과 같습니다:  1.       2
첫째 아이와 둘째 아이의 나이 차이는 가족의 상황과 개인적인 선호도에 따라 다를 수 있습니      2
야민 정음이란 한글 자모를 모양이 비슷한 것으로 바꾸어 단어를 다르게 표현하는 인터넷 밈입      2
인스타그램 스토리에 음악을 추가하는 방법은 다음과 같습니다:  1. 인스타그램 앱을 열고,      2
한국 현대 문학에 대한 일반 대중의 관심을 높이기 위해서는 다음과 같은 전략을 취할 수 있      2
야민 정음이란 한글 자모를 모양이 비슷한 것으로 바꾸어 단어를 다르게 표현하는 인터넷 밉입      2
축구 경기 중 부상을 최소화하기 위해서는 다음과 같은 조치를 취할 수 있습니다:  1. 적      2
인터넷 광고와 텔레비전 광고의 주요 차이점은 다음과 같습니다: 

In [29]:
def head_prefix(text, n=60):
    t = (text or "").strip().replace("\n"," ")
    return t[:n]

prefixes = sft_df["answer_norm"].apply(lambda x: head_prefix(x, 60))
top_prefix = prefixes.value_counts().head(30)
top_prefix


,count
answer_norm,
"죄송합니다. 특정 개인에 대한 질문에는 답변하기 어려우며, 개인정보 보호와 윤리적인 이유로 인해 해당 정보를",119
"죄송합니다. 저는 인공지능 챗봇으로 일반적인 지식을 통해 학습되어, 주관적인 판단이 개입되는 정보에 대해서는",11
양자 상호작용은 양자역학에서 다양한 방법으로 묘사될 수 있습니다. 다음은 주요한 세 가지 방법입니다: 1,2
피부 미백을 위한 제품에는 다양한 종류가 있습니다. 아래는 일반적으로 사용되는 피부 미백 제품의 몇 가지 예,2
클래식 음악의 접근성을 높이기 위해서는 다음과 같은 방법을 고려할 수 있습니다: 1. 교육 및 정보 제공:,2
야민 정음이란 한글 자모를 모양이 비슷한 것으로 바꾸어 단어를 다르게 표현하는 인터넷 밈입니다. 대한민국의,2
야민 정음이란 한글 자모를 모양이 비슷한 것으로 바꾸어 단어를 다르게 표현하는 인터넷 밉입니다. 대한민국의,2
망원경은 빛을 수집하기 위해 주로 두 가지 방법을 사용합니다. 1. 개구경 (Objective lens),1
자동차 기술에서 인공 지능과 머신 러닝은 매우 중요한 역할을 합니다. 인공 지능은 주로 운전 지원 시스템,1


In [30]:
for p in top_prefix.index[:10]:
    print("\n=== PREFIX ===", p)
    print(sft_df.loc[prefixes==p, "answer_norm"].iloc[0][:400])



=== PREFIX === 죄송합니다. 특정 개인에 대한 질문에는 답변하기 어려우며, 개인정보 보호와 윤리적인 이유로 인해 해당 정보를
죄송합니다. 특정 개인에 대한 질문에는 답변하기 어려우며, 개인정보 보호와 윤리적인 이유로 인해 해당 정보를 제공할 수 없습니다. 다른 주제나 일반적인 정보에 대한 질문은 언제든지 환영하니 자유롭게 물어보세요.

=== PREFIX === 죄송합니다. 저는 인공지능 챗봇으로 일반적인 지식을 통해 학습되어, 주관적인 판단이 개입되는 정보에 대해서는
죄송합니다. 저는 인공지능 챗봇으로 일반적인 지식을 통해 학습되어, 주관적인 판단이 개입되는 정보에 대해서는 답을 할 수 없습니다.

=== PREFIX === 양자 상호작용은 양자역학에서 다양한 방법으로 묘사될 수 있습니다. 다음은 주요한 세 가지 방법입니다:   1
양자 상호작용은 양자역학에서 다양한 방법으로 묘사될 수 있습니다. 다음은 주요한 세 가지 방법입니다: 

1. 해밀 토니 안 (Hamiltonian): 양자 상호작용은 해밀 토니 안 연산자를 사용하여 묘사됩니다. 해밀 토니 안은 시스템의 에너지와 운동량을 설명하는 연산자로, 양자역학적인 시간 발전을 결정하는 주요한 역할을 합니다. 해밀 토니 안은 시스템의 특성을 나타내는 행렬 형태로 표현됩니다. 

2. 상호작용 헤일리 연산자 (Interaction Hamiltonian): 양자 시스템 간의 상호작용은 상호작용 헤일리 연산자를 사용하여 묘사될 수 있습니다. 이 연산자는 시스템 간의 상호작용에 의해 발생하는 에너지를 나타내며, 해밀 토니 안에 추가되는 형태로 표현됩니다. 상호작용 헤일리 연산자는 시스템 간의 

=== PREFIX === 피부 미백을 위한 제품에는 다양한 종류가 있습니다. 아래는 일반적으로 사용되는 피부 미백 제품의 몇 가지 예
피부 미백을 위한 제품에는 다양한 종류가 있습니다. 아래는 일반적으로 사용되는 피부 미백 제품의 몇 가지 예시입니다:

1. 화이트닝 크림: 피부색을 밝게 하고 잡티를 개선하기 

In [33]:
#1)거절답변 등
SENSITIVE_Q_PATTERNS = [
    r"\b(전화번호|휴대폰|번호)\b",
    r"\b(주소|사는\s?곳|집\s?주소|거주지)\b",
    r"\b(주민등록|주민번호|신분증|여권)\b",
    r"\b(계좌|카드\s?번호|비밀번호|OTP|인증번호)\b",
    r"\b(메일|이메일)\b",
    r"\b(실명|이름|본명)\b",
    r"\b(카톡|카카오톡|인스타|SNS)\b",
    r"\b(병력|질병|진단|처방)\b",
    r"\b(소송|법률\s?상담|고소|고발)\b",
    r"(개인정보|프라이버시|사생활|신상)",
    r"(유출|보안|해킹|계정\s?탈취|피싱|스미싱)",
]

REFUSAL_A_PATTERNS = [
    r"(죄송|답변하기\s?어렵|제공할\s?수\s?없|도와드릴\s?수\s?없)",
    r"(개인정보|프라이버시|사생활)",
    r"(윤리적|정책|규정|가이드라인|보호)",
]

sens_q_re = re.compile("|".join(SENSITIVE_Q_PATTERNS))
ref_a_re  = re.compile("|".join(REFUSAL_A_PATTERNS))

def is_sensitive_question(q: str) -> int:
    q = (q or "")
    return int(bool(sens_q_re.search(q)))

def is_refusal_answer(a: str) -> int:
    a = (a or "")
    return int(bool(ref_a_re.search(a)))

# 2) SFT에 태그 추가 (question_norm / answer_norm 기준)
sft_df["is_sensitive_q"] = sft_df["question_norm"].apply(is_sensitive_question)
sft_df["is_refusal_a"]   = sft_df["answer_norm"].apply(is_refusal_answer)

# "정상 거절" = 질문이 민감 + 답이 거절/정책
sft_df["is_sensitive_refusal"] = ((sft_df["is_sensitive_q"] == 1) & (sft_df["is_refusal_a"] == 1)).astype(int)

# 3) 기존 avoid_flag랑 같이 보기
# sft_df["avoid_flag"] = sft_df["answer_norm"].apply(has_avoid)

print("SFT rows:", len(sft_df))
print("avoid ratio:", float(sft_df["avoid_flag"].mean()) if "avoid_flag" in sft_df else "no avoid_flag")
print("sensitive_q ratio:", sft_df["is_sensitive_q"].mean())
print("refusal_a ratio:", sft_df["is_refusal_a"].mean())
print("sensitive_refusal ratio:", sft_df["is_sensitive_refusal"].mean())

# 4) 핵심: '회피' 중에서 정상 거절 vs 쓸데없는 회피 분리
if "avoid_flag" in sft_df:
    avoid = sft_df[sft_df["avoid_flag"] == 1]
    print("\n--- within avoid ---")
    print("avoid count:", len(avoid))
    print("sensitive_refusal within avoid:", avoid["is_sensitive_refusal"].mean())
    print("non-sensitive avoid ratio(=bad-avoid-ish):", (avoid["is_sensitive_refusal"] == 0).mean())

# 5) 샘플 확인(정상 거절 TOP / 비민감 회피 TOP)
print("\n[Sample sensitive_refusal]")
display(sft_df[sft_df["is_sensitive_refusal"]==1][["question_norm","answer_norm"]].head(5))

if "avoid_flag" in sft_df:
    print("\n[Sample avoid but NOT sensitive_refusal]")
    display(sft_df[(sft_df["avoid_flag"]==1) & (sft_df["is_sensitive_refusal"]==0)][["question_norm","answer_norm"]].head(5))


SFT rows: 10580
avoid ratio: 0.23686200378071834
sensitive_q ratio: 0.006332703213610586
refusal_a ratio: 0.22164461247637052
sensitive_refusal ratio: 0.0029300567107750472

--- within avoid ---
avoid count: 2506
sensitive_refusal within avoid: 0.004788507581803671
non-sensitive avoid ratio(=bad-avoid-ish): 0.9952114924181963

[Sample sensitive_refusal]


,question_norm,answer_norm
199,올림픽 대회의 안전과 보안에 어떤 조치가 취해지나요?,올림픽 대회의 안전과 보안을 위해 다양한 조치가 취해집니다. 이러한 조치들은 주최국...
617,임상시험 참가자의 개인정보는 어떻게 보호되나요?,임상시험 참가자의 개인정보는 일반적으로 다음과 같은 방법으로 보호됩니다: \n\n1...
790,인터넷 보안과 개인 정보 보호는 어떻게 유지되나요?,인터넷 보안과 개인 정보 보호는 여러 가지 방법을 통해 유지됩니다. 주요한 방법은 ...
2451,무선 네트워크를 보호하기 위한 보안 조치는 어떤 것이 있나요?,무선 네트워크 보호를 위한 일반적인 보안 조치는 다음과 같습니다:\n\n1. 암호화...
2468,세계 복지 제도가 식량 보안을 어떻게 촉진하는가?,세계 복지 제도는 식량 보안을 촉진하기 위해 다음과 같은 방법을 사용할 수 있습니다...



[Sample avoid but NOT sensitive_refusal]


,question_norm,answer_norm
0,제주도 사투리와 한국어 간의 의사소통에 어떤 어려움이 있을까요?,제주도 사투리와 표준 한국어 사이에는 몇 가지 어려움이 있을 수 있습니다. \n\n...
2,콜레스테롤이 높으면 나쁜가요?,"네, 콜레스테롤 수치가 높으면 건강에 나쁜 영향을 미치는 경우가 많습니다. 콜레스테..."
4,언제부터 사람들이 생선을 먹기 시작했죠?,"물고기를 먹기 시작한 시기는 정확히 알려져 있지는 않지만, 지구상에서 가장 오래된 ..."
8,마취 중 어떻게 환자의 생명을 지키는가?,"마취 중 생명을 보호하는 주요 방법은 다음과 같습니다:\n\n1. 마취 과정 전, ..."
11,마취 중 근육 이완과 마비는 어떻게 달성되는가?,마취 중에 근육 이완과 마비는 다양한 방법으로 달성됩니다. 가장 일반적인 방법은 약...


In [34]:
# RM 신뢰도 EDA: (1) 완전성 (2) rank1 vs rank5 유사도 (3) 이상치(짧은 rank1 / 질문반복)
import numpy as np
import pandas as pd
import difflib

# df 가정: columns = ["data_id","rank","question_norm","answer_norm", ...]
# rank는 1~5 정수

# ----------------------------
# 0) 기본 sanity: rank 분포/완전성(1~5 다 있는지)
# ----------------------------
# data_id별로 rank set 확인
rank_counts = df.groupby("data_id")["rank"].nunique()
missing_any_ratio = (rank_counts < 5).mean()
print("data_id 수:", df["data_id"].nunique())
print("rank 1~5 모두 있는 data_id 비율:", 1 - missing_any_ratio)
print("rank 누락 data_id 개수:", int((rank_counts < 5).sum()))

# 어떤 rank가 자주 빠지는지
expected = set([1,2,3,4,5])
missing_rank_list = []
for did, sub in df.groupby("data_id"):
    got = set(sub["rank"].tolist())
    miss = sorted(list(expected - got))
    if miss:
        missing_rank_list.append([did, miss])

missing_rank_df = pd.DataFrame(missing_rank_list, columns=["data_id","missing_ranks"])
print("missing_rank_df shape:", missing_rank_df.shape)
display(missing_rank_df.head(10))

# ----------------------------
# 1) rank1 vs rank5 유사도 높은데 랭킹 다른 케이스 뽑기
# ----------------------------
def sim(a, b):
    a = (a or "").strip()
    b = (b or "").strip()
    if not a or not b:
        return np.nan
    return difflib.SequenceMatcher(None, a, b).ratio()

pairs = []
for did, sub in df.groupby("data_id"):
    if not ((sub["rank"]==1).any() and (sub["rank"]==5).any()):
        continue
    q  = sub["question_norm"].iloc[0]
    a1 = sub.loc[sub["rank"]==1, "answer_norm"].iloc[0]
    a5 = sub.loc[sub["rank"]==5, "answer_norm"].iloc[0]
    pairs.append([did, sim(a1,a5), len(a1 or ""), len(a5 or ""), q, a1[:200], a5[:200]])

sim_df = pd.DataFrame(
    pairs,
    columns=["data_id","sim_1v5","len_rank1","len_rank5","question_norm","rank1_head","rank5_head"]
).sort_values("sim_1v5", ascending=False)

print("sim_df shape:", sim_df.shape)
display(sim_df.head(20))

SIM_TH = 0.95
high_sim = sim_df[sim_df["sim_1v5"] >= SIM_TH].copy()
print(f"high_sim (sim>={SIM_TH}) count:", len(high_sim))
display(high_sim.head(20))


# ----------------------------
# 2) rank1이 너무 짧은 이상치
# ----------------------------
SHORT_TH = 60
rank1 = df[df["rank"]==1].copy()
rank1["a_len"] = rank1["answer_norm"].fillna("").astype(str).str.len()

short_rank1 = rank1[rank1["a_len"] < SHORT_TH][["data_id","question_norm","answer_norm","a_len"]].sort_values("a_len")
print(f"short_rank1 (<{SHORT_TH}) count:", len(short_rank1))
display(short_rank1.head(30))

# ----------------------------
# 3) rank1이 질문을 그대로 반복하는 이상치(프록시)
# ----------------------------
def repeats_question(q, a):
    q = (q or "").strip()
    a = (a or "").strip()
    if len(q) < 10 or len(a) < 10:
        return False
    # 질문 앞 20자가 답변 앞 80자에 포함되면 반복으로 간주(프록시)
    return q[:20] in a[:80]

rank1["q_repeat"] = rank1.apply(lambda r: repeats_question(r["question_norm"], r["answer_norm"]), axis=1)
qrepeat_rank1 = rank1[rank1["q_repeat"]==True][["data_id","question_norm","answer_norm"]]
print("qrepeat_rank1 count:", len(qrepeat_rank1))
display(qrepeat_rank1.head(30))




data_id 수: 26408
rank 1~5 모두 있는 data_id 비율: 1.0
rank 누락 data_id 개수: 0
missing_rank_df shape: (0, 2)


,data_id,missing_ranks


sim_df shape: (26408, 7)


,data_id,sim_1v5,len_rank1,len_rank5,question_norm,rank1_head,rank5_head
25781,f9f1786a-fe96-4d64-9220-92bed9fa27f2,1.0,30,30,반말로 질문을 해도 돼?,"네, 반말로 질문해도 괜찮습니다. 무엇을 도와드릴까요?","네, 반말로 질문해도 괜찮습니다. 무엇을 도와드릴까요?"
20932,caf4e5fe-50ba-4c01-8b07-d3bba9403dbb,1.0,23,23,재미있는 책을 추천해 줄래?,물론이죠! 어떤 장르의 책을 좋아하시나요?,물론이죠! 어떤 장르의 책을 좋아하시나요?
1777,110a52ec-6167-4ef9-b77c-156a31e5537d,1.0,25,25,"""일본어 공부를 하고 있습니다.""를 영어로 알려주세요.","""I am studying Japanese.""","""I am studying Japanese."""
22820,dd4050ba-114b-4ba4-bd85-e28d515e92fa,1.0,14,14,태국의 수도는 어디인가요?,태국의 수도는 방콕입니다.,태국의 수도는 방콕입니다.
12097,74c6ddd4-d9ac-4f04-b45d-03af06898d25,1.0,56,56,"""이 도시에 어떤 유명한 관광지가 있어요?""를 영어로 어떻게 질문하나요?","""What are some famous tourist attractions in t...","""What are some famous tourist attractions in t..."
7209,45289c86-88ca-49b2-a83a-24fb838faf30,1.0,33,33,남아프리카 공화국의 주요 도시 중 하나는 어디인가요?,남아프리카 공화국의 주요 도시 중 하나는 요하네스버그입니다.,남아프리카 공화국의 주요 도시 중 하나는 요하네스버그입니다.
23815,e6cc0bd7-9950-4e97-a91c-2afbb6669116,1.0,46,46,"""이 도시의 맛집은 어디에 있어요?""를 영어로 알려 줘.","""Where are the best restaurants in this city?""","""Where are the best restaurants in this city?"""
8601,524e24bd-22ae-4def-8de2-6381e2667a30,1.0,21,21,선물은 영어로 어떻게 씁니까?,"선물은 영어로 ""gift""라고 씁니다.","선물은 영어로 ""gift""라고 씁니다."
12922,7d086104-2f75-433d-a079-4e279a517a27,1.0,27,27,유리의 원료로 쓰이는 비금속의 명칭을 쓰시오.,유리의 원료로 쓰이는 비금속의 명칭은 규소입니다.,유리의 원료로 쓰이는 비금속의 명칭은 규소입니다.
3538,21c276fc-fb29-4e33-a636-26c7e8e1bd05,1.0,44,44,정확한 영어 문장을 입력해 주세요.,Please provide an accurate English sentence.,Please provide an accurate English sentence.


high_sim (sim>=0.95) count: 42


,data_id,sim_1v5,len_rank1,len_rank5,question_norm,rank1_head,rank5_head
25781,f9f1786a-fe96-4d64-9220-92bed9fa27f2,1.0,30,30,반말로 질문을 해도 돼?,"네, 반말로 질문해도 괜찮습니다. 무엇을 도와드릴까요?","네, 반말로 질문해도 괜찮습니다. 무엇을 도와드릴까요?"
20932,caf4e5fe-50ba-4c01-8b07-d3bba9403dbb,1.0,23,23,재미있는 책을 추천해 줄래?,물론이죠! 어떤 장르의 책을 좋아하시나요?,물론이죠! 어떤 장르의 책을 좋아하시나요?
1777,110a52ec-6167-4ef9-b77c-156a31e5537d,1.0,25,25,"""일본어 공부를 하고 있습니다.""를 영어로 알려주세요.","""I am studying Japanese.""","""I am studying Japanese."""
22820,dd4050ba-114b-4ba4-bd85-e28d515e92fa,1.0,14,14,태국의 수도는 어디인가요?,태국의 수도는 방콕입니다.,태국의 수도는 방콕입니다.
12097,74c6ddd4-d9ac-4f04-b45d-03af06898d25,1.0,56,56,"""이 도시에 어떤 유명한 관광지가 있어요?""를 영어로 어떻게 질문하나요?","""What are some famous tourist attractions in t...","""What are some famous tourist attractions in t..."
7209,45289c86-88ca-49b2-a83a-24fb838faf30,1.0,33,33,남아프리카 공화국의 주요 도시 중 하나는 어디인가요?,남아프리카 공화국의 주요 도시 중 하나는 요하네스버그입니다.,남아프리카 공화국의 주요 도시 중 하나는 요하네스버그입니다.
23815,e6cc0bd7-9950-4e97-a91c-2afbb6669116,1.0,46,46,"""이 도시의 맛집은 어디에 있어요?""를 영어로 알려 줘.","""Where are the best restaurants in this city?""","""Where are the best restaurants in this city?"""
8601,524e24bd-22ae-4def-8de2-6381e2667a30,1.0,21,21,선물은 영어로 어떻게 씁니까?,"선물은 영어로 ""gift""라고 씁니다.","선물은 영어로 ""gift""라고 씁니다."
12922,7d086104-2f75-433d-a079-4e279a517a27,1.0,27,27,유리의 원료로 쓰이는 비금속의 명칭을 쓰시오.,유리의 원료로 쓰이는 비금속의 명칭은 규소입니다.,유리의 원료로 쓰이는 비금속의 명칭은 규소입니다.
3538,21c276fc-fb29-4e33-a636-26c7e8e1bd05,1.0,44,44,정확한 영어 문장을 입력해 주세요.,Please provide an accurate English sentence.,Please provide an accurate English sentence.


short_rank1 (<60) count: 2487


,data_id,question_norm,answer_norm,a_len
1950,03f3b1b2-deb4-4c74-a59b-3c4d5dfac0fa,진화론이 뭐야?,진화론이 뭐야?,8
75315,91fb26a9-1d26-4a11-a85e-80c6786a2020,공자는 누구인가요?,공자는 누구인가요?,10
52480,64dfe23a-dbbf-4b43-97d3-b7b3d6212d3d,바로크 시대와 로맨틱 시대의 클래식 음악은 어떤 차이점을 보이나요?,사실 확인이 어려움,10
102675,c7102943-b578-4985-b0ee-2607f856a82a,"토크란 무엇이며, 어떻게 계산하나요?",사실 확인이 어려움,10
114270,dda2ca11-0734-4e34-b8c7-0a55067b0d87,껌은 언제 개발됐어?,껌은 언제 개발됐어?,11
86275,a6e840fd-d1a3-4bde-944b-35de053701b2,SM의 역사를 알려줘.,SM의 역사를 알려줘.,12
16600,1fb198fe-68e9-4689-8e02-38287c90458f,마스킹에 대해 알려줘.,마스킹에 대해 알려줘.,12
84855,a445e8ad-9bfd-4624-b273-be98cbc671b8,웃긴 외국릴스 찾아줘.,웃긴 외국릴스 찾아줘.,12
112410,da143ee4-73b2-4461-83a0-37412a981806,최초의 SNS는 뭐야?,최초의 SNS는 뭐야?,12
62795,79613cd3-c426-4779-a9b1-499e0a30f956,일본 노래 추천해 줘.,일본 노래 추천해 줘.,12


qrepeat_rank1 count: 6550


,data_id,question_norm,answer_norm
10,0009bc91-2f88-4591-bd88-874df8f388a7,현대 철학은 과거의 철학 전통과 어떻게 연결되고 있어?,현대 철학은 과거의 철학 전통과 어떻게 연결되고 있어?
20,0010463b-7378-4870-881e-f37272968b52,부모가 자녀에게 균형 잡힌 식사 습관을 가르치는 방법에 대해 알려주세요.,균형 잡힌 식사 습관을 가르치는 것은 자녀의 건강과 웰빙에 매우 중요합니다. 아래는...
25,00138710-5779-477e-84ff-3b4c315242f2,사회적 책임을 갖춘 의류 브랜드를 만들려면 어떻게 해야 할까요?,사회적 책임을 갖춘 의류 브랜드를 만들기 위해서는 다음과 같은 단계를 따를 수 있습...
35,001e0148-2b37-42ae-8b76-be5ac80a3cd3,가장 중요한 국제 기후 변화 문제 중 하나는 뭔지 알려 줄 수 있어?,가장 중요한 국제 기후 변화 문제 중 하나는 온실가스 배출입니다. 온실가스는 인간의...
50,002335c0-dbb2-44b7-bc97-0fe9df2c805a,금광과 다른 광물 자원 채굴 비교에서 어떤 차이가 있나요?,금광과 다른 광물 자원 채굴 비교에서는 몇 가지 차이점이 있습니다.\n\n1. 가치...
75,0028da46-fc25-4581-8f76-3c26dc1c4b6d,단백질-리기 그랜드 상호 작용을 연구하는 방법이 뭐야?,단백질-리기 그랜드 상호 작용을 연구하는 방법은 여러 가지가 있습니다. 주로 사용되...
85,002d07ce-6a50-4fe5-8db5-368996910909,오디오의 무음 부분을 자를 수 있는 프로그램을 추천해 주세요.,오디오의 무음 부분을 자를 수 있는 몇 가지 유용한 프로그램을 추천해 드릴게요.\n...
90,00306295-847c-4949-9b0d-5126a74273ff,이 지역에서 발견된 지질학적 발견은 어떤 중요성을 가지고 있나요?,이 지역에서 발견된 지질학적 발견은 여러 가지 중요성을 가질 수 있습니다. \n\n...
125,003ccfa1-3d59-45a3-ac6e-bbe30e5f7d61,도덕적 판단에서 공공 이익과 개인 이익의 상충이 발생하면 어떻게 대처해야 할지 서술...,"도덕적 판단에서 공공 이익과 개인 이익의 상충이 발생했을 때, 다음과 같은 방법으로..."
150,0057b7ce-6547-4cad-93eb-2f51a25fbfa1,다문화 가정에서 자라나는 아이들에게 대한 교육 시스템에 대해서 알고 싶어요.,다문화 가정에서 자라나는 아이들에게 대한 교육 시스템은 해당 국가의 교육 정책과 다...


In [35]:
!pip -q install transformers sentencepiece

from transformers import AutoTokenizer

MODEL_NAMES = {
    "EXAONE-4.0-1.2B": "LGAI-EXAONE/EXAONE-4.0-1.2B",
    "KANANA-1.5-8B-Instruct-2505": "kakaocorp/kanana-1.5-8b-instruct-2505",
}

def token_len_factory(tok):
    def _token_len(text: str) -> int:
        return len(tok.encode(text or "", add_special_tokens=False))
    return _token_len

def show_percentiles(arr, name):
    ps = np.percentile(arr, [50, 90, 95, 99, 99.5]).astype(int)
    print(f"[{name}] p50/p90/p95/p99/p99.5 =", ps)

def trunc_rate(arr, L):
    arr = np.asarray(arr)
    return float((arr > L).mean())

for tag, model_id in MODEL_NAMES.items():
    print("\n" + "="*90)
    print("Tokenizer:", tag, "|", model_id)

    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    token_len = token_len_factory(tok)

    # --- SFT: prompt + response ---
    sft_prompt_tok = sft_df["question_norm"].astype(str).apply(token_len).values
    sft_resp_tok   = sft_df["answer_norm"].astype(str).apply(token_len).values
    sft_total_tok  = sft_prompt_tok + sft_resp_tok

    show_percentiles(sft_total_tok, f"{tag} | SFT total(prompt+resp)")
    for L in [512, 768, 1024, 1536, 2048]:
        print(f"{tag} | SFT trunc@{L}:", trunc_rate(sft_total_tok, L))

    # --- RM: 대표로 rank1만 (질문+답변) ---
    rm_rank1 = df[df["rank"]==1].copy()
    rm_prompt_tok = rm_rank1["question_norm"].astype(str).apply(token_len).values
    rm_resp_tok   = rm_rank1["answer_norm"].astype(str).apply(token_len).values
    rm_total_tok  = rm_prompt_tok + rm_resp_tok

    show_percentiles(rm_total_tok, f"{tag} | RM(rank1) total(prompt+resp)")
    for L in [512, 768, 1024, 1536, 2048]:
        print(f"{tag} | RM trunc@{L}:", trunc_rate(rm_total_tok, L))

    # --- PPO: 질문만 ---
    ppo_prompt_tok = ppo_df["question_norm"].astype(str).apply(token_len).values
    show_percentiles(ppo_prompt_tok, f"{tag} | PPO prompt only")
    for L in [128, 256, 384, 512, 768, 1024]:
        print(f"{tag} | PPO trunc@{L}:", trunc_rate(ppo_prompt_tok, L))




Tokenizer: EXAONE-4.0-1.2B | LGAI-EXAONE/EXAONE-4.0-1.2B


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

[EXAONE-4.0-1.2B | SFT total(prompt+resp)] p50/p90/p95/p99/p99.5 = [333 451 484 554 591]
EXAONE-4.0-1.2B | SFT trunc@512: 0.0277882797731569
EXAONE-4.0-1.2B | SFT trunc@768: 0.0001890359168241966
EXAONE-4.0-1.2B | SFT trunc@1024: 0.0
EXAONE-4.0-1.2B | SFT trunc@1536: 0.0
EXAONE-4.0-1.2B | SFT trunc@2048: 0.0
[EXAONE-4.0-1.2B | RM(rank1) total(prompt+resp)] p50/p90/p95/p99/p99.5 = [327 467 511 764 847]
EXAONE-4.0-1.2B | RM trunc@512: 0.04888670099969706
EXAONE-4.0-1.2B | RM trunc@768: 0.009769766737352317
EXAONE-4.0-1.2B | RM trunc@1024: 0.0012874886398061194
EXAONE-4.0-1.2B | RM trunc@1536: 0.0001514692517418964
EXAONE-4.0-1.2B | RM trunc@2048: 0.0001514692517418964
[EXAONE-4.0-1.2B | PPO prompt only] p50/p90/p95/p99/p99.5 = [15 21 23 29 35]
EXAONE-4.0-1.2B | PPO trunc@128: 0.0002751247887434658
EXAONE-4.0-1.2B | PPO trunc@256: 0.0
EXAONE-4.0-1.2B | PPO trunc@384: 0.0
EXAONE-4.0-1.2B | PPO trunc@512: 0.0
EXAONE-4.0-1.2B | PPO trunc@768: 0.0
EXAONE-4.0-1.2B | PPO trunc@1024: 0.0

Tokeni

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

[KANANA-1.5-8B-Instruct-2505 | SFT total(prompt+resp)] p50/p90/p95/p99/p99.5 = [387 524 562 647 678]
KANANA-1.5-8B-Instruct-2505 | SFT trunc@512: 0.12296786389413988
KANANA-1.5-8B-Instruct-2505 | SFT trunc@768: 0.0012287334593572778
KANANA-1.5-8B-Instruct-2505 | SFT trunc@1024: 0.0
KANANA-1.5-8B-Instruct-2505 | SFT trunc@1536: 0.0
KANANA-1.5-8B-Instruct-2505 | SFT trunc@2048: 0.0
[KANANA-1.5-8B-Instruct-2505 | RM(rank1) total(prompt+resp)] p50/p90/p95/p99/p99.5 = [375 532 582 861 965]
KANANA-1.5-8B-Instruct-2505 | RM trunc@512: 0.13124810663435324
KANANA-1.5-8B-Instruct-2505 | RM trunc@768: 0.015184792487125114
KANANA-1.5-8B-Instruct-2505 | RM trunc@1024: 0.0032944562253862464
KANANA-1.5-8B-Instruct-2505 | RM trunc@1536: 0.0003029385034837928
KANANA-1.5-8B-Instruct-2505 | RM trunc@2048: 3.78673129354741e-05
[KANANA-1.5-8B-Instruct-2505 | PPO prompt only] p50/p90/p95/p99/p99.5 = [18 26 28 35 40]
KANANA-1.5-8B-Instruct-2505 | PPO trunc@128: 0.0002751247887434658
KANANA-1.5-8B-Instruct-25

In [39]:
# =========================
# Dataset release exporter (SFT / RM / PPO)
# - stable split by data_id
# - tokenizer-based clipping (prompt/response token budget)
# - outputs:
#   dataset_release/
#     split_ids/train_ids.txt val_ids.txt test_ids.txt
#     sft/sft_{split}.jsonl
#     rm/rm_pair_{split}.jsonl
#     ppo/ppo_prompt_{split}.jsonl
#     meta/stats.json README.md
# =========================

import os, json, hashlib
import numpy as np
import pandas as pd

from transformers import AutoTokenizer

# -------------------------
# 0) Config
# -------------------------
OUT_ROOT = "dataset_release"

MODEL_NAME_FOR_CLIP = "kakaocorp/kanana-1.5-8b-instruct-2505"
# MODEL_NAME_FOR_CLIP = "LGAI-EXAONE/EXAONE-4.0-1.2B"

tok = AutoTokenizer.from_pretrained(MODEL_NAME_FOR_CLIP, use_fast=True)

# 1024로 잡으면 대부분 안 잘림
MAX_TOTAL_TOK = 1024      # prompt+response 합
MAX_PROMPT_TOK = 256      # prompt만 최대
MAX_PPO_PROMPT_TOK = 256  # PPO prompt only

SPLIT_RATIOS = (0.90, 0.05, 0.05)  # train/val/test

# -------------------------
# 1) Utils: stable split
# -------------------------
def stable_hash_int(x: str) -> int:
    h = hashlib.md5(str(x).encode("utf-8")).hexdigest()
    return int(h[:8], 16)

def split_of_id(data_id: str, ratios=SPLIT_RATIOS) -> str:
    r = stable_hash_int(data_id) / 0xFFFFFFFF
    tr, va, te = ratios
    if r < tr:
        return "train"
    elif r < tr + va:
        return "val"
    else:
        return "test"

def add_split_col(df_, id_col="data_id"):
    df_ = df_.copy()
    df_["split"] = df_[id_col].astype(str).apply(split_of_id)
    return df_

# -------------------------
# 2) Token clipping helpers
# -------------------------
def _encode(text: str):
    return tok.encode(text or "", add_special_tokens=False)

def _decode(ids):
    return tok.decode(ids, skip_special_tokens=True)

def clip_text_to_tokens(text: str, max_tok: int) -> str:
    ids = _encode(text)
    if len(ids) <= max_tok:
        return text if text is not None else ""
    return _decode(ids[:max_tok])

def clip_prompt_response(prompt: str, response: str, max_prompt_tok: int, max_total_tok: int):
    """
    Always returns (prompt_clipped, response_clipped) exactly 2 strings.
    """
    p = clip_text_to_tokens(str(prompt), max_prompt_tok)

    p_ids = _encode(p)
    budget = max_total_tok - len(p_ids)
    if budget < 0:
        p = _decode(p_ids[:max_total_tok])
        return p, ""

    r_ids = _encode(str(response))
    r = _decode(r_ids[:max(0, budget)])
    return p, r

# -------------------------
# 3) Prepare split ids (union of ids across datasets)
# -------------------------
def collect_all_ids(sft_df, ppo_df, rm_df):
    ids = []
    if "data_id" in sft_df.columns:
        ids.append(sft_df["data_id"].astype(str))
    if "data_id" in ppo_df.columns:
        ids.append(ppo_df["data_id"].astype(str))
    if "data_id" in rm_df.columns:
        ids.append(rm_df["data_id"].astype(str))
    all_ids = pd.concat(ids, ignore_index=True).dropna().unique().tolist()
    return all_ids

all_ids = collect_all_ids(sft_df, ppo_df, df)
id_split = pd.DataFrame({"data_id": all_ids})
id_split["split"] = id_split["data_id"].apply(split_of_id)

# -------------------------
# 4) Make directories
# -------------------------
dirs = [
    f"{OUT_ROOT}/split_ids",
    f"{OUT_ROOT}/sft",
    f"{OUT_ROOT}/rm",
    f"{OUT_ROOT}/ppo",
    f"{OUT_ROOT}/meta",
]
for d in dirs:
    os.makedirs(d, exist_ok=True)

# save split id lists
for sp in ["train","val","test"]:
    path = f"{OUT_ROOT}/split_ids/{sp}_ids.txt"
    ids_sp = id_split.loc[id_split["split"]==sp, "data_id"].tolist()
    with open(path, "w", encoding="utf-8") as f:
        f.write("\n".join(ids_sp))

# -------------------------
# 5) Export SFT jsonl
#    format: {"data_id":..., "prompt":..., "response":...}
# -------------------------
sft_use = sft_df[["data_id","question_norm","answer_norm"]].copy()
sft_use["data_id"] = sft_use["data_id"].astype(str)
sft_use = sft_use.merge(id_split, on="data_id", how="left")

# make prompt/response and clip
def _clip_sft_row(r):
    p, a = clip_prompt_response(r["question_norm"], r["answer_norm"], MAX_PROMPT_TOK, MAX_TOTAL_TOK)
    return pd.Series([p, a])

tmp = sft_use.apply(_clip_sft_row, axis=1)
tmp.columns = ["prompt", "response"]
sft_use["prompt"] = tmp["prompt"]
sft_use["response"] = tmp["response"]

for sp in ["train","val","test"]:
    out_path = f"{OUT_ROOT}/sft/sft_{sp}.jsonl"
    sub = sft_use[sft_use["split"]==sp]
    with open(out_path, "w", encoding="utf-8") as f:
        for _, r in sub.iterrows():
            f.write(json.dumps(
                {"data_id": r["data_id"], "prompt": r["prompt"], "response": r["response"]},
                ensure_ascii=False
            ) + "\n")

# -------------------------
# 6) Export RM pair jsonl
#    기본: rank1(chosen) vs rank5(rejected)
#    format: {"data_id":..., "prompt":..., "chosen":..., "rejected":...}
# -------------------------
rm = df[["data_id","rank","question_norm","answer_norm"]].copy()
rm["data_id"] = rm["data_id"].astype(str)
rm = rm.merge(id_split, on="data_id", how="left")

# pick rank1 & rank5
r1 = rm[rm["rank"]==1].rename(columns={"answer_norm":"chosen"})[["data_id","question_norm","chosen","split"]]
r5 = rm[rm["rank"]==5].rename(columns={"answer_norm":"rejected"})[["data_id","rejected"]]
rm_pair = r1.merge(r5, on="data_id", how="inner")

def _clip_rm_row(r):
    p, ch = clip_prompt_response(r["question_norm"], r["chosen"], MAX_PROMPT_TOK, MAX_TOTAL_TOK)
    p2, rj = clip_prompt_response(p, r["rejected"], MAX_PROMPT_TOK, MAX_TOTAL_TOK)
    return pd.Series([p, ch, rj])

tmp = rm_pair.apply(_clip_rm_row, axis=1)
tmp.columns = ["prompt","chosen","rejected"]
rm_pair["prompt"] = tmp["prompt"]
rm_pair["chosen"] = tmp["chosen"]
rm_pair["rejected"] = tmp["rejected"]

for sp in ["train","val","test"]:
    out_path = f"{OUT_ROOT}/rm/rm_pair_{sp}.jsonl"
    sub = rm_pair[rm_pair["split"]==sp]
    with open(out_path, "w", encoding="utf-8") as f:
        for _, r in sub.iterrows():
            f.write(json.dumps(
                {"data_id": r["data_id"], "prompt": r["prompt"], "chosen": r["chosen"], "rejected": r["rejected"]},
                ensure_ascii=False
            ) + "\n")

# -------------------------
# 7) Export PPO prompt jsonl
#    format: {"data_id":..., "prompt":...}
# -------------------------
ppo_use = ppo_df[["data_id","question_norm"]].copy()
ppo_use["data_id"] = ppo_use["data_id"].astype(str)
ppo_use = ppo_use.merge(id_split, on="data_id", how="left")

ppo_use["prompt"] = ppo_use["question_norm"].astype(str).apply(lambda x: clip_text_to_tokens(x, MAX_PPO_PROMPT_TOK))

for sp in ["train","val","test"]:
    out_path = f"{OUT_ROOT}/ppo/ppo_prompt_{sp}.jsonl"
    sub = ppo_use[ppo_use["split"]==sp]
    with open(out_path, "w", encoding="utf-8") as f:
        for _, r in sub.iterrows():
            f.write(json.dumps(
                {"data_id": r["data_id"], "prompt": r["prompt"]},
                ensure_ascii=False
            ) + "\n")

# -------------------------
# 8) Meta stats.json + README.md
# -------------------------
def count_lines(path):
    with open(path, "r", encoding="utf-8") as f:
        return sum(1 for _ in f)

stats = {
    "clip_tokenizer": MODEL_NAME_FOR_CLIP,
    "max_total_tok": MAX_TOTAL_TOK,
    "max_prompt_tok": MAX_PROMPT_TOK,
    "max_ppo_prompt_tok": MAX_PPO_PROMPT_TOK,
    "splits": {
        sp: {
            "ids": len(id_split[id_split["split"]==sp]),
            "sft": count_lines(f"{OUT_ROOT}/sft/sft_{sp}.jsonl"),
            "rm_pair": count_lines(f"{OUT_ROOT}/rm/rm_pair_{sp}.jsonl"),
            "ppo_prompt": count_lines(f"{OUT_ROOT}/ppo/ppo_prompt_{sp}.jsonl"),
        } for sp in ["train","val","test"]
    }
}

with open(f"{OUT_ROOT}/meta/stats.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

readme = f"""# dataset_release

This release contains split-consistent datasets for SFT / RM / PPO.

## Split rule
- stable hash by `data_id` (md5 prefix) -> train/val/test = {SPLIT_RATIOS}

## Files
- split_ids/{{train,val,test}}_ids.txt
- sft/sft_{{train,val,test}}.jsonl
  - fields: data_id, prompt, response
- rm/rm_pair_{{train,val,test}}.jsonl
  - fields: data_id, prompt, chosen, rejected
  - chosen=rank1, rejected=rank5
- ppo/ppo_prompt_{{train,val,test}}.jsonl
  - fields: data_id, prompt

## Clipping (token-based)
- tokenizer: {MODEL_NAME_FOR_CLIP}
- max_total_tok(prompt+response): {MAX_TOTAL_TOK}
- max_prompt_tok: {MAX_PROMPT_TOK}
- max_ppo_prompt_tok: {MAX_PPO_PROMPT_TOK}

See meta/stats.json for counts.
"""
with open(f"{OUT_ROOT}/meta/README.md", "w", encoding="utf-8") as f:
    f.write(readme)

print("Export complete:", OUT_ROOT)
print(json.dumps(stats, ensure_ascii=False, indent=2))


✅ Export complete: dataset_release
{
  "clip_tokenizer": "kakaocorp/kanana-1.5-8b-instruct-2505",
  "max_total_tok": 1024,
  "max_prompt_tok": 256,
  "max_ppo_prompt_tok": 256,
  "splits": {
    "train": {
      "ids": 56273,
      "sft": 9514,
      "rm_pair": 23859,
      "ppo_prompt": 22900
    },
    "val": {
      "ids": 3115,
      "sft": 536,
      "rm_pair": 1288,
      "ppo_prompt": 1291
    },
    "test": {
      "ids": 3043,
      "sft": 530,
      "rm_pair": 1261,
      "ppo_prompt": 1252
    }
  }
}


In [40]:
!zip -r dataset_release.zip dataset_release


  adding: dataset_release/ (stored 0%)
  adding: dataset_release/ppo/ (stored 0%)
  adding: dataset_release/ppo/ppo_prompt_test.jsonl (deflated 61%)
  adding: dataset_release/ppo/ppo_prompt_train.jsonl (deflated 61%)
  adding: dataset_release/ppo/ppo_prompt_val.jsonl (deflated 61%)
  adding: dataset_release/meta/ (stored 0%)
  adding: dataset_release/meta/README.md (deflated 46%)
  adding: dataset_release/meta/stats.json (deflated 57%)
  adding: dataset_release/sft/ (stored 0%)
  adding: dataset_release/sft/sft_test.jsonl (deflated 70%)
  adding: dataset_release/sft/sft_val.jsonl (deflated 69%)
  adding: dataset_release/sft/sft_train.jsonl (deflated 70%)
  adding: dataset_release/rm/ (stored 0%)
  adding: dataset_release/rm/rm_pair_val.jsonl (deflated 73%)
  adding: dataset_release/rm/rm_pair_train.jsonl (deflated 74%)
  adding: dataset_release/rm/rm_pair_test.jsonl (deflated 73%)
  adding: dataset_release/split_ids/ (stored 0%)
  adding: dataset_release/split_ids/train_ids.txt (deflat

In [ ]:
import os, glob, json
import pandas as pd

BASE = "/content/drive/MyDrive/train"
SFT_GLOB = BASE + "/**/*sft*.jsonl"
RM_GLOB  = BASE + "/**/*rm*.jsonl"
PPO_GLOB = BASE + "/**/*ppo*.jsonl"

def count_lines(fp):
    n = 0
    with open(fp, "r", encoding="utf-8") as f:
        for _ in f:
            n += 1
    return n

def count_rows_any(fp):
    ext = os.path.splitext(fp)[1].lower()
    if ext == ".jsonl":
        return count_lines(fp)
    if ext == ".csv":
        return sum(1 for _ in open(fp, "r", encoding="utf-8")) - 1  # header 제외
    if ext in [".parquet", ".pq"]:
        return len(pd.read_parquet(fp))

    if ext == ".json":
        with open(fp, "r", encoding="utf-8") as f:
            obj = json.load(f)
        return len(obj) if isinstance(obj, list) else 1
    raise ValueError(f"지원하지 않는 확장자: {fp}")

def scan_and_count(path_glob, title):
    files = sorted(set(glob.glob(path_glob, recursive=True)))
    if not files:
        print(f"\n[{title}] 파일을 못 찾았어. glob를 더 정확히 지정해줘.\n  glob={path_glob}")
        return
    total = 0
    print(f"\n[{title}] found {len(files)} files")
    for fp in files:
        try:
            n = count_rows_any(fp)
            total += n
            print(f"  - {fp}  -> {n}")
        except Exception as e:
            print(f"  - {fp}  -> ERROR: {e}")
    print(f"[{title}] TOTAL rows = {total}")

scan_and_count(SFT_GLOB, "SFT 원본")
scan_and_count(RM_GLOB,  "RM 원본")
scan_and_count(PPO_GLOB, "PPO 원본")
